# Backpropagation in depth

In the [last lesson](https://github.com/VikParuchuri/zero_to_gpt/blob/master/explanations/rnn.ipynb), we learned how to create a recurrent neural network.  We now know how to build several network architectures using components like dense layers, softmax, and recurrent layers.

We've been a bit loose with how we cover backpropagation, to make neural network architecture easier to understand.  In this lesson, we'll do a deep dive into how backpropagation works.  We'll do this by building a computational graph that keeps track of the different operations that transform the input data.

# The Softmax function

In a previous lesson, we introduced the softmax function.  This is used to convert the output of a neural network into probabilities that can be used to make predictions.  The softmax function is defined as:

$$\zeta=\frac{e^{\hat{y_{i}}}}{\sum_{j=0}e^{\hat{y_{j}}}}$$

For each row of our neural network output, we raise $e$ to the power of our output value, then divide by the sum of $e$ raised to the power of each of the outputs for that row.

The softmax function looks like this in code:

In [29]:
import numpy as np

def softmax_func(normalized):
    raised = np.exp(normalized)
    output = raised / np.sum(raised, axis=1).reshape(-1,1)
    return output

We can test the softmax function out using some fake data that we generate:

In [30]:
# 5 rows and 3 columns of random numbers
x = np.random.rand(5, 3)

# Generate random correct labels for later
# Exactly one label per row will be correct
y = np.zeros_like(x)
inds = (np.arange(0,y.shape[0]), np.random.randint(0, 3, size=y.shape[0]))
y[inds] = 1

In [31]:
normalized = x - np.max(x, axis=-1).reshape(-1,1)
softmax_func(normalized)

array([[0.27790738, 0.35198167, 0.37011095],
       [0.40952508, 0.40890145, 0.18157347],
       [0.35959571, 0.33292964, 0.30747465],
       [0.17993713, 0.44686784, 0.37319503],
       [0.27006585, 0.4142223 , 0.31571185]])

In the above code, we subtract the maximum from each element in the row before passing the data into the softmax function.  This prevents numerical underflow or overflow.  Each [numeric type](https://numpy.org/doc/stable/user/basics.types.html) (float, integer, etc) can only hold a certain number of digits.  For example, floating point 16 can store 5 exponent bits, and ten digit bits (each bit is only base 2, so this is less than the same number of base-10 digits).  The maximum value we can store in `float16` is `65500`:

In [10]:
# Check the maximum value we can assign to float16
np.finfo('float16').max

65500.0

In [11]:
# This is an example of numeric overflow, where we store more digits than float16 can hold
a = np.array([0], dtype=np.float16)
a[0] = 6.55e5

/var/folders/xz/9z84c__j28g8tg28bmcthjj00000gn/T/ipykernel_82094/2542651828.py:3: RuntimeWarning: overflow encountered in cast
  a[0] = 6.55e5


When we raise $e$ to a very large or small number, we can generate a number that is too large to store in our specific data type.  Subtracting the max gives us the same end result, but reduces the risk of overflow.  Feel free to try the softmax out with and without subtracting the max to see how it works!

# Staged Softmax

Instead of computing the softmax derivative, we previously used the fact that the derivative of the softmax and negative log likelihood functions "cancel out", and end up with a derivative of $p-y$.  But what if we want to find the derivative ourselves?

We can approach it analytically, and find the derivative of the entire function.  But an easier method is to break the softmax function apart into individual operations.  Each operation will make a single modification to the data:

![](images/comp_graph/softmax_steps.svg)

We perform 3 operations on the data:

- Exp - we raise e to the power x.
- Sum - we add up the $e^x$ values for each row.
- Divide - we divide the $e^x$ values by the sums.

Note that the output of `Exp` is passed to both the `Sum` and `Divide` operations.

By breaking up the softmax this way, we can take the derivative of each individual piece instead of the whole function at once.  By the [chain rule](https://www.khanacademy.org/math/ap-calculus-ab/ab-differentiation-2-new/ab-3-1a/a/chain-rule-review), multiplying the derivative of each individual operation will result in the derivative of the whole function.

Now we can build the forward pass of our staged softmax.  The derivative of multiplication is easier to calculate than division, so we'll swap some of our operations to remove the division.

Luckily for us, raising a value `x` to the power `-1` is the same as taking `1/x`.  So instead of dividing `Exp/Sum`, we can do `Exp * Sum ^ -1$, leaving us with these operations:

- Exp
- Sum
- Pow - we invert the sum by raising to the power `-1`
- Multiply - we multiply the inverted sum and the exp values

In [20]:
raised = np.exp(x) # step 1
summed = np.sum(raised, axis=-1).reshape(-1,1) # step 2
pow = summed ** -1 # step 3
softmax = raised * pow # step 4

In [21]:
softmax

array([[0.1998043 , 0.40974354, 0.39045216],
       [0.47895933, 0.27306597, 0.2479747 ],
       [0.3358449 , 0.39191405, 0.27224105],
       [0.23924132, 0.22612083, 0.53463785],
       [0.33206489, 0.2672699 , 0.40066521]])

Our staged softmax has the exact same output as our original function, but is broken up into steps where we apply individual operations.

# Softmax Derivative

To get the derivative of the softmax, we need to reverse the operations we did before:

![](images/comp_graph/softmax_steps_full_bwd.svg)

We will take in the derivative of the loss with respect to the output of the softmax.  Then each operation will compute the derivative of the loss with respect to the input operations.  Note that `Exp` is input to two operations, so it will sum both gradients.

To calculate loss, we'll use negative log likelihood, which is $NLL = - \sum_{i=0} y_{i} \log p_{i}$.  Since $y$ is only non-zero at one position per row, this will only have a single term (basically $-y_{i} * \log p_{i}$ where $i$ is the correct label where $y$ equals `1`).

We'll use the negative log likelihood derivative below:

In [28]:
def nll_grad(y, pred):
    return -1 * y / pred

loss_grad = nll_grad(y, softmax)

We can then calculate the softmax derivative by multiplying the derivatives of the individual operations.  Remember that a derivative is the rate of change of a function as we change the input.

- Exp - the derivative of $e^x$ is $e^x$ (this is a very cool property of $e$!)
- Sum - since a sum operation will combine input elements into one, we just distribute the gradient over all input elements.  A change to any of the input elements will have a direct impact on the output.
- Pow - the derivative of $x^{-1}$ is $-1 * x^{-2}$.  More on this [here](https://www.khanacademy.org/math/old-ap-calculus-ab/ab-derivative-rules/ab-diff-negative-fraction-powers/a/power-rule-review).
- Multiply - the derivatives of $x*y$ are $y$ wrt $x$ and $x$ wrt $y$.  This is because any change to $x$ is multiplied by $y$, and vice versa.  Thus the rate of change of $x$ is $y$, and vice versa.

We can now create the backward pass of our staged softmax.  The backward pass will start with the loss gradient.  This will be a matrix showing how much we need to adjust each of the output values from our softmax to reduce our loss.  We can then compute gradients for each operation, ending with the gradient against the input, `x`.  If `x` was the output of a neural network, we would continue backpropagation at that point to adjust the network parameters.

We'll name each gradient according to the step it is a gradient for, not the step it is coming from.  So `raised_grad` is the gradient on `raised`.

In [ ]:
# Step 4 derivative
raised_grad = loss_grad * pow
pow_grad = loss_grad * raised
pow_grad = np.sum(pow_grad, axis=-1).reshape(-1,1) # reshape gradient to match input data

# Step 3
summed_grad = (-1 * summed ** -2) * pow_grad

# Step 2
raised_grad_2 = np.ones_like(raised) * summed_grad # distribute gradient across inputs

# Step 1
raised_grad += raised_grad_2 # sum incoming gradients
x_grad = raised_grad * np.exp(x)

We did two things above that might be confusing.  The first is that we summed 2 gradients on raised.  This is because raised connects to 2 operations, and both have separate gradients.  Whenever this happens, we sum the gradients.

The second is that we reshaped `pow_grad` to have a single column.  This is to match `pow`, which only had `1` column in the forward pass.  Whenever a gradient doesn't match the shape of the input data, we change the size of the gradient to match it.  This is because the gradient represents the partial derivative against each parameter in the input data.